In [120]:
import requests
import pandas as pd
from typing import Literal

class CountriesHook:
    raw_data: list[dict]
    
    def __init__(self) -> None:
        pass

    def get_countries(self) -> list[dict]:
        '''
        en : Returns a list of dicts with all countries listed in rest_countries api
        pt_br: Retorna um lista de dicionários com todos os países listados na api rest_countries
        '''
        URL = 'https://restcountries.com/v3.1/all'
        self.raw_data = requests.get(url = URL).json()
        return  self.raw_data




class Country:
    fields_to_parse: list = ['name', 'independent', 'unMember', 'currencies', 'capital', 'region', 
                             'subregion', 'languages', 'translations', 'latlng', 'landlocked', 'borders', 'area', 
                             'demonyms', 'flag', 'maps', 'population', 'gini', 'car', 'timezones', 'continents', 'flags', 
                             'startOfWeek', 'capitalInfo', 'postalCode', 'coatOfArms']
    lang_table: pd.DataFrame
    def __init__(self, country_record) -> None:
        self.country_record = country_record
    
    
    def print_fields(self):
        '''en: Print all available fields from a country
        pt_br: Printa todos os campos de um pais
        '''
        for i in self.fields_to_parse: print(f'{i}: {self.country_record[i]}')


    def make_table_from_dict(self, field:str) -> pd.DataFrame:
        '''
        en: Makes a table from a dict, the key will be the name of the columns and the value will be the value inside the cell. Return a one line pd.DataFrame
        pt_br: Faz uma tabela a partir de um dicionário, a chave será o nome da coluna e o valor será o valor da célula. Retorn um pd.DataFrame de tamanho 1
        '''
        if not isinstance(self.country_record[field], dict): raise TypeError('O valor do campo informado não é um dicionário')
        country_record = {i: [self.country_record[field][i]] for i in self.country_record[field] if not isinstance(self.country_record[field][i], dict)}
        return pd.DataFrame(country_record)
        
    def make_key_value_table_from_dict(self, field:str, columns=['col1', 'col2'])-> pd.DataFrame:
        '''
        en: Makes a table from a dict, where every key is a columns and the values are another column. Column names can be especified
        pt_br: Faz uma tabela a partir de um dicionário, onde todas as chaves são uma coluna e os valores são outra.  Os nome de coluna podem ser especificados
        '''
        assert len(columns) == 2, 'Informe somente duas colunas. len(columns) must be 2.'
        _dict = self.country_record[field]
        return  pd.DataFrame({
                    columns[0]: [i for i in _dict],
                    columns[1]: [_dict[i] for i in _dict]

                })
        
    def make_table_from_list(country_record:dict, field:str):
        if not isinstance(country_record[field], list): raise TypeError('O valor do campo informado não é um lista')
        dic = {field: country_record[field]}   
        dic['common']  = [country_record['name']['common'] for i in range(len(country_record[field]))]
        return pd.DataFrame(dic)[['common', field]]

    def get_country_name(self, cols:Literal['common', 'official', None]=None):
        if cols is None: return self.make_table_from_dict('name')
        if isinstance(cols, str): cols = [cols]
        return self.make_table_from_dict('name')[cols]
    
    def create_language_table(self):
      '''
      en: creates a fact table from languages
      '''
      langs = self.make_key_value_table_from_dict('languages', ['lang_code', 'lang'])
      name = self.get_country_name('common')
      self.lang_table = name.merge(langs, how='cross')
      return self.lang_table



In [ ]:
hook = CountriesHook()
hook.get_countries()


,common,lang_code,lang
0,Guam,cha,Chamorro
1,Guam,eng,English
2,Guam,spa,Spanish
